In [1]:
from enhanced_lib.exchange import ExchangeCache,Database

db = Database('http://localhost:8000')

# await db.generate_and_save_future_trades('main_account','BTCUSDT','gbozee1_sub_account')
exchange = await db.get_initialized_exchange('main_account','BTCUSDT','gbozee1_sub_account')
# exchange.open_orders.get_sl_orders('short',True)
await exchange.account.get_trades(exchange.symbol)
# config = exchange.config 


# exchange.get_next_tradable_zone('long',True)
# config.determine_optimum_risk('long',{'stop': 36909.8, 'entry': 40762.1792, 'size': 0.1}
#                               ,max_size=.1,gap=.1,with_trades=True,no_of_cpu=1)

# future_instance.get_trading_zones('long')
# future_instance.config.long_liquidation_price
# future_instance.config.as_dict()
# zones = future_instance.config.get_trading_zones('short')
# zones
# stop = future_instance.config.long_liquidation_price
# stop
# app_config = future_instance.config.app_config
# app_config.entry = 44779.3
# app_config.stop = stop
# # app_config.raw = True
# # workers.determine_optimum_reward(app_config)
# exchange.maximum_sell_size
# exchange.get_zone_for_position('long')
# exchange.get_zone_for_position('long')
# exchange.get_next_tradable_zone('long')
# exchange.config_params_for_future_trades2('long',True)

{'long': [{'stop': 36481.7,
   'entry': 41335.7992,
   'size': 0.4,
   'risk_reward': 193,
   'risk_per_trade': 281,
   'trades': [{'entry': 36497.4,
     'risk': 1.456,
     'quantity': 0.093,
     'sell_price': 41335.7992,
     'incurred_sell': 43232.5,
     'stop': 36481.7,
     'pnl': 1607.742,
     'fee': 2.715,
     'net': 900.284,
     'incurred': 626.365,
     'stop_percent': 0.0,
     'rr': 193,
     'avg_entry': 37025.5,
     'avg_size': 0.373,
     'start_entry': 41335.7992},
    {'entry': 36522.6,
     'risk': 1.456,
     'quantity': 0.036,
     'sell_price': 41335.7992,
     'incurred_sell': 53834.1,
     'stop': 36481.7,
     'pnl': 1157.8,
     'fee': 1.052,
     'net': 347.59,
     'incurred': 623.214,
     'stop_percent': 0.001,
     'rr': 193,
     'avg_entry': 37200.8,
     'avg_size': 0.28,
     'start_entry': 41335.7992},
    {'entry': 36547.8,
     'risk': 1.456,
     'quantity': 0.022,
     'sell_price': 41335.7992,
     'incurred_sell': 64732.4,
     'stop': 364

In [1]:
from enhanced_lib.calculations.shared import build_config
from enhanced_lib.calculations.workers.optimum_risk_reward import eval_func,determine_optimum_reward,run_in_parallel
from enhanced_lib.calculations.workers.utils import chunks_in_threads,run_in_threads
app_config = future_instance.config.app_config
app_config.entry = 41993.9
app_config.stop = 44779.3
app_config.kind = 'short'
app_config.risk_reward = 80
# app_config.risk_per_trade = 70
print('app_config',app_config)
# app_config.support = 41900
# app_config.resistance  = 45000
# signal = build_config(app_config,{
#     # 'raw_instance':True,
#     'decimal_places':app_config.decimal_places,
#     'increase':True,
#     'risk_reward':80,
#     'risk_per_trade': 6,
#     'entry': 44200,
#     'kind': 'long',
#     'stop': 42000,
#     'currentEntry':44200
# })
# signal
# # signal.get_bulk_trade_zones(44200,kind='long')
app_config.strategy = 'entry'
# eval_func(80,app_config)
# for i in range(30,199,1):
#     uu = eval_func(i,app_config)
#     # print('uu',len(uu['result']))
# func = run_in_parallel(eval_func, [(x,app_config) for x in range(30,199,1)],no_of_cpu=8)
# func
# bb = chunks_in_threads(eval_func, [(x,app_config) for x in range(30,199,1)],no_of_cpu=4,num_threads=40)
# len(bb)
# func = run_in_threads(eval_func, [(x,app_config) for x in range(30,199,1)],num_threads=2)
# determine_optimum_reward(app_config)
# exchange.future_instance.config.long_liquidation_price
# exchange.get_next_tradable_zone('long')
# exchange.get_next_tradable_zone('short')
exchange.config_params_for_future_trades('short',True)

NameError: name 'future_instance' is not defined